In [1]:
import requests

import numpy as np
import pandas as pd

from pycoingecko import CoinGeckoAPI

from bs4 import BeautifulSoup as bts

### Why isn't this a script?

We are only generating the dataset once. We don't need to write a script that does that is properly formatted and tested, we can do it using a notebooks and test our results at the end.

## Market cap

### Using the Coin Gecko API

First we use the CG API to get the first page of results becasue of the nature of the distribution of market caps (hard to define a rule that will encapsulate all because of a skewed distribution).

In [2]:
cg = CoinGeckoAPI()

In [7]:
cg.get_coins_markets(vs_currency="usd")

[{'id': 'bitcoin',
  'symbol': 'btc',
  'name': 'Bitcoin',
  'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400',
  'current_price': 60877,
  'market_cap': 1200383561114,
  'market_cap_rank': 1,
  'fully_diluted_valuation': 1278410169130,
  'total_volume': 22680479122,
  'high_24h': 61670,
  'low_24h': 60041,
  'price_change_24h': -447.0383883457398,
  'price_change_percentage_24h': -0.72898,
  'market_cap_change_24h': -7774407834.871826,
  'market_cap_change_percentage_24h': -0.64349,
  'circulating_supply': 19718284.0,
  'total_supply': 21000000.0,
  'max_supply': 21000000.0,
  'ath': 73738,
  'ath_change_percentage': -17.42963,
  'ath_date': '2024-03-14T07:10:36.635Z',
  'atl': 67.81,
  'atl_change_percentage': 89689.98199,
  'atl_date': '2013-07-06T00:00:00.000Z',
  'roi': None,
  'last_updated': '2024-06-29T08:30:00.613Z'},
 {'id': 'ethereum',
  'symbol': 'eth',
  'name': 'Ethereum',
  'image': 'https://coin-images.coingecko.com/coins/images/27

In [9]:
# Make a dataframe with all available coins
api_df = pd.DataFrame(cg.get_coins_markets(vs_currency="usd")) 

In [10]:
api_df.head(3)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,60877.000000,1200383561114,1,1.278410e+12,2.268048e+10,61670.000000,...,2.100000e+07,21000000.0,73738.00,-17.42963,2024-03-14T07:10:36.635Z,67.810000,89689.98199,2013-07-06T00:00:00.000Z,None,2024-06-29T08:30:00.613Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,3389.850000,407408413310,2,4.074084e+11,1.096509e+10,3461.390000,...,1.201847e+08,NaN,4878.26,-30.46746,2021-11-10T14:24:19.604Z,0.432979,783305.11856,2015-10-20T00:00:00.000Z,"{'times': 73.44475449637534, 'currency': 'btc'...",2024-06-29T08:30:24.276Z
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,0.998468,112638701925,3,1.126387e+11,3.731959e+10,0.999714,...,1.128115e+11,NaN,1.32,-24.53815,2018-07-24T00:00:00.000Z,0.572521,74.39223,2015-03-02T00:00:00.000Z,None,2024-06-29T08:25:26.340Z


### Scraping

Only have market cap data up to page 47, and we can skip the first page.

In [53]:
scrape_dict_list = []
for page_num in range(2, 47):
    # Get HTML for the page
    url = 'https://www.coingecko.com/?page='+ str(page_num)
    result = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    # Get list of names and tickers
    raw_name_list = soup.findAll('div', class_='tw-text-gray-700 dark:tw-text-moon-100 tw-font-semibold tw-text-sm tw-leading-5')
    name_ticker_list = [raw_name.text.split() for raw_name in raw_name_list][1:-1] # [1:] to skip "Highlights"
    # Get list of market caps
    if page_num == 2:
        start_mc = 800000000
    else:
        start_mc = scrape_dict_list[-1]['market_cap']
    raw_price_list = soup.findAll('span', {'data-price-target': 'price'})
    mc_list = [start_mc]
    for i in range(10, len(raw_price_list)):
        num_item = float(raw_price_list[i].text.replace('$', '').replace(',', ''))
        if mc_list[-1] > num_item > mc_list[-1]*0.8:
            mc_list.append(num_item)
    mc_list = mc_list[1:] # [1:] to skip start_mc
    # Combine into a dictionary and append to list
    if len(name_ticker_list) <= len(mc_list):
        scrape_len = len(name_ticker_list)
    else:
        scrape_len = len(mc_list)
    for i in range(scrape_len):
        coin_dict={}
        coin_dict['symbol'] = name_ticker_list[i][-1]
        coin_dict['name'] = ' '.join(name_ticker_list[i][:-1])
        coin_dict['market_cap'] = mc_list[i]
        scrape_dict_list.append(coin_dict)

In [54]:
scrape_dict_list

[{'symbol': 'AGIX', 'name': 'SingularityNET', 'market_cap': 782508693.0},
 {'symbol': 'MSOL', 'name': 'Marinade Staked SOL', 'market_cap': 776535873.0},
 {'symbol': 'XTZ', 'name': 'Tezos', 'market_cap': 772011345.0},
 {'symbol': 'SAND', 'name': 'The Sandbox', 'market_cap': 768372355.0},
 {'symbol': 'USDD', 'name': 'USDD', 'market_cap': 733001688.0},
 {'symbol': 'GNO', 'name': 'Gnosis', 'market_cap': 719684362.0},
 {'symbol': 'FTN', 'name': 'Fasttoken', 'market_cap': 701020890.0},
 {'symbol': 'BOME', 'name': 'BOOK OF MEME', 'market_cap': 698968660.0},
 {'symbol': 'RON', 'name': 'Ronin', 'market_cap': 697590323.0},
 {'symbol': 'WLD', 'name': 'Worldcoin', 'market_cap': 694842691.0},
 {'symbol': 'MOG', 'name': 'Mog Coin', 'market_cap': 693841346.0},
 {'symbol': 'CFX', 'name': 'Conflux', 'market_cap': 684563688.0},
 {'symbol': 'ZRO', 'name': 'LayerZero', 'market_cap': 677073302.0},
 {'symbol': 'CHZ', 'name': 'Chiliz', 'market_cap': 671590422.0},
 {'symbol': 'FRXETH', 'name': 'Frax Ether', '

In [56]:
scrape_dict_list[-1]

{'symbol': 'VST', 'name': 'Vesta Stable', 'market_cap': 1137.4}

In [55]:
len(scrape_dict_list)

2099

In [57]:
# Make a dataframe with all available coins
scrape_df = pd.DataFrame(scrape_dict_list) 

### Combine API data with scraped data

In [89]:
# Transform API df
api_df ['symbol_upper'] = api_df['symbol'].apply(lambda x: x.upper())
api_trans_df = api_df[['symbol_upper', 'name', 'market_cap']]
api_trans_df = api_trans_df.rename(columns={"symbol_upper": "symbol"})

In [70]:
# Transform scrape df
scrape_trans_df = scrape_df.astype({'market_cap': 'int64'})

In [81]:
# Union - should be disjoint sets
combined_df = pd.concat([api_trans_df, scrape_trans_df], axis=0)

In [92]:
# Duplicates caused by same symbol on different chain
scrape_trans_df[scrape_trans_df['symbol'] == 'WOLF']

,symbol,name,market_cap
218,WOLF,LandWolf,141191119
428,WOLF,LandWolf,53375438
1804,WOLF,moonwolf.io,61956


In [93]:
# Dropping duplicates to avoid confusion
clean_combined_df = combined_df.drop_duplicates(subset=['symbol'], keep=False)

In [94]:
clean_combined_df

,symbol,name,market_cap
0,BTC,Bitcoin,1200383561114
1,ETH,Ethereum,407408413310
2,USDT,Tether,112638701925
3,BNB,BNB,87612827137
4,SOL,Solana,65481910665
...,...,...,...
2094,YARD,Solyard Finance,1506
2095,DBL,Doubloon,1504
2096,DEXSHARE,dexSHARE,1445
2097,COWRIE,Cowrie,1235


## Whitepaper info